In [49]:
import http.client, json

conn = http.client.HTTPSConnection("api.scansan.com")
headers = {"X-Auth-Token": "48d455cc-963d-4c4c-8588-2aadd2a7f2dd"}

conn.request("GET", "/v1/district/NW1/growth", headers=headers)
res = conn.getresponse()
body = res.read()

if res.status != 200:
    print(f"HTTP {res.status} {res.reason}")
    print(body.decode("utf-8", errors="replace"))
else:
    data = json.loads(body or b"{}")
    print(json.dumps(data, indent=2))


{
  "area_code": "NW1",
  "area_code_type": "area_code_district",
  "response_time": "0.00",
  "data": {
    "monthly_data": [],
    "yearly_data": [
      {
        "area_code_district": "NW1",
        "year_month": "2019",
        "avg_price": 955011,
        "avg_price_percentage_change": null,
        "median_price": 655000,
        "median_price_percentage_change": null,
        "source": "property_transactions"
      },
      {
        "area_code_district": "NW1",
        "year_month": "2020",
        "avg_price": 866169,
        "avg_price_percentage_change": -9.3,
        "median_price": 660000,
        "median_price_percentage_change": 0.8,
        "source": "property_transactions"
      },
      {
        "area_code_district": "NW1",
        "year_month": "2021",
        "avg_price": 1004730,
        "avg_price_percentage_change": 16,
        "median_price": 678000,
        "median_price_percentage_change": 2.7,
        "source": "property_transactions"
      },
      {
     

In [48]:
import http.client, json

conn = http.client.HTTPSConnection("api.scansan.com")
headers = {"X-Auth-Token": "48d455cc-963d-4c4c-8588-2aadd2a7f2dd"}

conn.request("GET", "/v1/kingscross/search", headers=headers)
res = conn.getresponse()
body = res.read()

if res.status != 200:
    print(f"HTTP {res.status} {res.reason}")
    print(body.decode("utf-8", errors="replace"))
else:
    data = json.loads(body or b"{}")
    print(json.dumps(data, indent=2))

HTTP 404 Not Found
404 page not found



In [ ]:
# London monthly postcode aggregation from HM Land Registry PPD
import pandas as pd
from pathlib import Path

ppd_path = Path("yearsold.csv")
output_dir = Path("data")
output_dir.mkdir(exist_ok=True)

columns = [
    "transaction_id",
    "price",
    "date_of_transfer",
    "postcode",
    "property_type",
    "old_new",
    "duration",
    "paon",
    "saon",
    "street",
    "locality",
    "town_city",
    "district",
    "county",
    "ppd_category",
    "record_status"
]

ppd = pd.read_csv(
    ppd_path,
    header=None,
    names=columns,
    dtype={
        "transaction_id": "string",
        "price": "Int64",
        "date_of_transfer": "string",
        "postcode": "string",
        "property_type": "string",
        "old_new": "string",
        "duration": "string",
        "paon": "string",
        "saon": "string",
        "street": "string",
        "locality": "string",
        "town_city": "string",
        "district": "string",
        "county": "string",
        "ppd_category": "string",
        "record_status": "string"
    }
)

ppd["postcode"] = ppd["postcode"].str.strip().str.upper()
ppd["county"] = ppd["county"].str.strip().str.upper()

london = ppd[ppd["county"] == "GREATER LONDON"].copy()
london["month"] = (
    pd.to_datetime(london["date_of_transfer"], errors="coerce")
    .dt.to_period("M")
    .astype(str)
)

monthly = (
    london.groupby(["postcode", "month"], as_index=False)
    .agg(
        median_price=("price", "median"),
        mean_price=("price", "mean"),
        sales=("price", "size")
    )
)

monthly_path = output_dir / "london_ppd_monthly.csv"
monthly.to_csv(monthly_path, index=False)
print(f"Wrote {monthly_path} with {len(monthly):,} rows")

# Optional: join coordinates if you have a postcode-to-lat/lon file
coords_path = output_dir / "postcode_coords.csv"  # expects columns: postcode, latitude, longitude
if coords_path.exists():
    coords = pd.read_csv(coords_path)
    coords["postcode"] = coords["postcode"].astype(str).str.strip().str.upper()
    merged = monthly.merge(coords, on="postcode", how="left")
    missing = merged["latitude"].isna().mean() * 100
    merged_path = output_dir / "london_ppd_monthly_coords.csv"
    merged.to_csv(merged_path, index=False)
    print(f"Wrote {merged_path}. Missing coords: {missing:.2f}%")
else:
    print("No postcode_coords.csv found in ./data — add it to get lat/lon joins.")


In [ ]:
# Build postcode -> lat/lon using Code-Point Open and join to monthly data
from pathlib import Path
import pandas as pd
from pyproj import Transformer

monthly_path = Path("data/london_ppd_monthly.csv")
codepo_dir = Path("codepo_gb/Data/CSV")
output_dir = Path("data")
output_dir.mkdir(exist_ok=True)

monthly = pd.read_csv(monthly_path)
monthly["postcode"] = monthly["postcode"].astype(str).str.strip().str.upper()
postcode_set = set(monthly["postcode"].unique())

cols = ["PC", "PQ", "EA", "NO", "CY", "RH", "LH", "CC", "DC", "WC"]
transformer = Transformer.from_crs(27700, 4326, always_xy=True)

records = []
for path in sorted(codepo_dir.glob("*.csv")):
    df = pd.read_csv(path, header=None, names=cols, dtype={"PC": "string"})
    df["PC"] = df["PC"].str.strip().str.upper()
    df = df[df["PC"].isin(postcode_set)]
    if df.empty:
        continue
    east = pd.to_numeric(df["EA"], errors="coerce").to_numpy()
    north = pd.to_numeric(df["NO"], errors="coerce").to_numpy()
    lon, lat = transformer.transform(east, north)
    out = pd.DataFrame({
        "postcode": df["PC"].values,
        "latitude": lat,
        "longitude": lon
    })
    records.append(out)

coords = pd.concat(records, ignore_index=True)
coords = coords.dropna(subset=["latitude", "longitude"])
coords = coords.drop_duplicates(subset=["postcode"]).reset_index(drop=True)
coords_path = output_dir / "postcode_coords.csv"
coords.to_csv(coords_path, index=False)

merged = monthly.merge(coords, on="postcode", how="left")
merged_path = output_dir / "london_ppd_monthly_coords.csv"
merged.to_csv(merged_path, index=False)

missing = merged["latitude"].isna().mean() * 100
print(f"Postcodes matched: {len(coords):,}")
print(f"Missing coords: {missing:.2f}%")
print(f"Wrote {coords_path} and {merged_path}")


In [ ]:
# Build postcode area GeoJSON per month using gb-postcodes polygons
from pathlib import Path
import json
import pandas as pd

areas_dir = Path("gb-postcodes/gb-postcodes-v5/areas")
monthly_path = Path("data/london_ppd_monthly.csv")

monthly = pd.read_csv(monthly_path)
monthly["area"] = monthly["postcode"].astype(str).str.extract(r"^([A-Z]+)")
area_month = (
    monthly.groupby(["area", "month"], as_index=False)
    .agg(
        median_price=("median_price", "median"),
        mean_price=("mean_price", "mean"),
        sales=("sales", "sum")
    )
)

months = sorted(area_month["month"].unique())
features_by_month = {m: [] for m in months}

for area_file in areas_dir.glob("*.geojson"):
    area = area_file.stem
    area_rows = area_month[area_month["area"] == area]
    if area_rows.empty:
        continue

    geo = json.loads(area_file.read_text())
    geom = None
    if geo.get("type") == "Feature":
        geom = geo.get("geometry")
    elif geo.get("type") == "FeatureCollection" and geo.get("features"):
        geom = geo["features"][0].get("geometry")

    if not geom:
        continue

    for _, row in area_rows.iterrows():
        month = row["month"]
        features_by_month[month].append({
            "type": "Feature",
            "properties": {
                "area": area,
                "median_price": float(row["median_price"]) if row["median_price"] == row["median_price"] else None,
                "mean_price": float(row["mean_price"]) if row["mean_price"] == row["mean_price"] else None,
                "sales": int(row["sales"]) if row["sales"] == row["sales"] else 0
            },
            "geometry": geom
        })

out_dir = Path("data/area_geojson")
out_dir.mkdir(exist_ok=True)

for month, feats in features_by_month.items():
    out_path = out_dir / f"area_{month}.geojson"
    out_path.write_text(json.dumps({"type": "FeatureCollection", "features": feats}))

print(f"Wrote {len(months)} area GeoJSON files to {out_dir}")
